In [7]:
import os
from pprint import pprint
from typing import Dict, Any

import requests
from crewai import Agent, Crew, LLM, Process, Task
from crewai.knowledge.source.base_knowledge_source import BaseKnowledgeSource
from crewai.knowledge.source.csv_knowledge_source import CSVKnowledgeSource
from crewai.knowledge.source.json_knowledge_source import JSONKnowledgeSource
from crewai.knowledge.source.pdf_knowledge_source import PDFKnowledgeSource
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
from crewai.knowledge.source.text_file_knowledge_source import TextFileKnowledgeSource
from pydantic import Field

In [7]:
policy_text = """
Our return policy allows customers to return any product within 30 days of purchase. 
Refunds will be issued only if the item is unused and in original packaging.
Customers must provide proof of purchase when requesting a return.
"""
return_policy_knowledge = StringKnowledgeSource(content=policy_text, chunk_size=4000, chunk_overlap=200)

llm = LLM(model="gpt-4o")

returns_agent = Agent(
    role="Product Returns Assistant",
    goal="Answer customer questions about return policy accurately.",
    backstory="You work in customer service and specialize in returns, refunds, and policies.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

returns_task = Task(
    description="Answer the customer question: {question}",
    expected_output="A concise and accurate answer.",
    agent=returns_agent
)

crew = Crew(
    agents=[returns_agent],
    tasks=[returns_task],
    process=Process.sequential,
    knowledge_sources=[return_policy_knowledge],
    embedder={
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-small",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_key": os.getenv("OPENAI_API_KEY")
        }
    },
    verbose=True
)

query = "Can I get a refund if I used the item once?"
result = crew.kickoff(inputs={"question": query})



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 56a93532-fe4b-49db-8fa6-de765610f32a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 📚 Retrieved Knowledge ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Additional Information:                                                                                        │
│  Our return policy allows customers to return any product within 30 days of purchase.                           │
│  Refunds will be issued only if the item is unused and in original packaging.                                   │
│  Customers must provide proof of purchase when requesting a return.                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Returns Assistant                                                                               │
│                                                                                                                 │
│  Task: Answer the customer question: Can I get a refund if I used the item once?                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Returns Assistant                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Refunds are only issued if the item is unused and in its original packaging as per our return policy. Since    │
│  the item has been used, unfortunately, it would not qualify for a refund. However, please ensure to have       │
│  proof of purchase for any further inquiries or concerns about returns.                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d5d1584f-2bbf-40f0-89af-343776e40d12                                                                     │
│  Agent: Product Returns Assistant                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 56a93532-fe4b-49db-8fa6-de765610f32a                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Refunds are only issued if the item is unused and in its original packaging as per our return    │
│  policy. Since the item has been used, unfortunately, it would not qualify for a refund. However, please        │
│  ensure to have proof of purchase for any further inquiries or concerns about returns.                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [8]:
pprint(result.raw)

('Refunds are only issued if the item is unused and in its original packaging '
 'as per our return policy. Since the item has been used, unfortunately, it '
 'would not qualify for a refund. However, please ensure to have proof of '
 'purchase for any further inquiries or concerns about returns.')


In [9]:
product_info = """
Our new product, Beacon AI, launches in Q3. It's a platform for automating enterprise workflows using AI agents. 
Key selling points: secure deployment, agent chaining, and seamless API integration.
"""
product_knowledge = StringKnowledgeSource(content=product_info)

llm = LLM(model="gpt-4o")

product_analyst = Agent(
    role="Product Launch Specialist",
    goal="Answer internal questions about our upcoming product.",
    backstory="You work on the go-to-market team and know the product details inside out.",
    knowledge_sources=[product_knowledge],
    embedder={
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-small",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_key": os.getenv("OPENAI_API_KEY")
        }
    },
    llm=llm,
    verbose=True
)

task = Task(
    description="What is Beacon AI and when does it launch?",
    expected_output="A short summary for internal use.",
    agent=product_analyst
)

crew = Crew(
    agents=[product_analyst],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
print(result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3daabbbb-10e6-4439-ac61-019f45d2580c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 📚 Retrieved Knowledge ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Additional Information:                                                                                        │
│  Our new product, Beacon AI, launches in Q3. It's a platform for automating enterprise workflows using AI       │
│  agents.                                                                                                        │
│  Key selling points: secure deployment, agent chaining, and seamless API integration.                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Launch Specialist                                                                               │
│                                                                                                                 │
│  Task: What is Beacon AI and when does it launch?                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Launch Specialist                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Beacon AI is our upcoming platform specifically designed for automating enterprise workflows with              │
│  cutting-edge AI agents. It offers key advantages such as secure deployment, agent chaining capabilities to     │
│  create complex workflow automations, and seamless API integration for efficient connectivity with existing     │
│  tools. Beacon AI is scheduled to launch in Q3, marking a pivotal step in enhancing workflow automation across  │
│  enterprises.                                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 290d526a-5294-426a-84e0-c4ab07ef7fef                                                                     │
│  Agent: Product Launch Specialist                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3daabbbb-10e6-4439-ac61-019f45d2580c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Beacon AI is our upcoming platform specifically designed for automating enterprise workflows     │
│  with cutting-edge AI agents. It offers key advantages such as secure deployment, agent chaining capabilities   │
│  to create complex workflow automations, and seamless API integration for efficient connectivity with existing  │
│  tools. Beacon AI is scheduled to launch in Q3, marking a pivotal step in enhancing workflow automation across  │
│  enterprises.                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Beacon AI is our upcoming platform specifically designed for automating enterprise workflows with cutting-edge AI agents. It offers key advantages such as secure deployment, agent chaining capabilities to create complex workflow automations, and seamless API integration for efficient connectivity with existing tools. Beacon AI is scheduled to launch in Q3, marking a pivotal step in enhancing workflow automation across enterprises.


In [10]:
product_info = """
Our new product, Beacon AI, launches in Q3. It's a platform for automating enterprise workflows using AI agents. 
Key selling points: secure deployment, agent chaining, and seamless API integration.
"""
product_knowledge = StringKnowledgeSource(content=product_info)

llm = LLM(model="gpt-4o")

marketing_agent = Agent(
    role="Marketing Strategist",
    goal="Craft messaging around our product launch.",
    backstory="You translate product features into compelling stories.",
    llm=llm,
    verbose=True
)

support_agent = Agent(
    role="Customer Support Specialist",
    goal="Prepare FAQs and answers for product-related customer inquiries.",
    backstory="You make sure support teams are ready before launch.",
    llm=llm,
    verbose=True
)

marketing_task = Task(
    description="Create a short headline summarizing what Beacon AI does.",
    expected_output="A 1-line marketing headline.",
    agent=marketing_agent
)

support_task = Task(
    description="Draft an FAQ entry answering: What is Beacon AI?",
    expected_output="A short FAQ-style answer for users.",
    agent=support_agent
)

crew = Crew(
    agents=[marketing_agent, support_agent],
    tasks=[marketing_task, support_task],
    knowledge_sources=[product_knowledge],
    embedder={
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-small",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_key": os.getenv("OPENAI_API_KEY")
        }
    },
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6b34b710-896b-4833-9ce8-341f2a5d2332                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 📚 Retrieved Knowledge ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Additional Information:                                                                                        │
│  Our new product, Beacon AI, launches in Q3. It's a platform for automating enterprise workflows using AI       │
│  agents.                                                                                                        │
│  Key selling points: secure deployment, agent chaining, and seamless API integration.                           │
│                                                                                                                 │
│                                                                                                                 │
│  Our return policy allows customers to return any product within 30 days of purchase.                           │
│  Refunds will be issued only if the item is unused and in original packaging.                                   │
│  Customers must provide proof of purchase when requesting a return.                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Marketing Strategist                                                                                    │
│                                                                                                                 │
│  Task: Create a short headline summarizing what Beacon AI does.                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Support Specialist                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **What is Beacon AI?**                                                                                         │
│  Beacon AI is an advanced platform designed to automate enterprise workflows securely using intelligent AI      │
│  agent chaining and seamless API integration. It enables businesses to optimize processes while ensuring        │
│  secure deployment for sensitive operations. Beacon AI is set to launch in Q3, offering cutting-edge solutions  │
│  for streamlined workflow automation.                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0d2d7816-4aca-4b69-a56a-aa06c847dd7b                                                                     │
│  Agent: Customer Support Specialist                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6b34b710-896b-4833-9ce8-341f2a5d2332                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **What is Beacon AI?**                                                                           │
│  Beacon AI is an advanced platform designed to automate enterprise workflows securely using intelligent AI      │
│  agent chaining and seamless API integration. It enables businesses to optimize processes while ensuring        │
│  secure deployment for sensitive operations. Beacon AI is set to launch in Q3, offering cutting-edge solutions  │
│  for streamlined workflow automation.                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

**What is Beacon AI?**  
Beacon AI is an advanced platform designed to automate enterprise workflows securely using intelligent AI agent chaining and seamless API integration. It enables businesses to optimize processes while ensuring secure deployment for sensitive operations. Beacon AI is set to launch in Q3, offering cutting-edge solutions for streamlined workflow automation.


In [13]:
text_source = TextFileKnowledgeSource(file_paths=["hr_policy.txt"])

llm = LLM(model="gpt-4o")

hr_agent = Agent(
    role="HR Policy Assistant",
    goal="Help employees understand HR guidelines.",
    backstory="You are an experienced HR support assistant, trained on company policy documents.",
    knowledge_sources=[text_source],
    embedder={
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-small",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_key": os.getenv("OPENAI_API_KEY")
        }
    },
    llm=llm
)

task = Task(
    description="What is the sick leave protocol for employees?",
    expected_output="A brief explanation of the sick leave process.",
    agent=hr_agent
)

crew = Crew(
    agents=[hr_agent],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3e61bef1-aa82-4a2d-91da-99a3f6e0cc16                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 📚 Retrieved Knowledge ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Additional Information: Company HR Policy - Leave and Attendance                                               │
│                                                                                                                 │
│  1. All full-time employees are entitled to 21 days of paid leave annually.                                     │
│  2. New employees are eligible for leave after completing their first 30 days.                                  │
│  3. Sick leave must be informed on the same day before 10 AM.                                                   │
│  4. Remote work is allowed up to 3 days a week with manager approval.                                           │
│  5. Any leave exceeding 3 consecutive days must be supported by documentation.                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: HR Policy Assistant                                                                                     │
│                                                                                                                 │
│  Task: What is the sick leave protocol for employees?                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: HR Policy Assistant                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The sick leave protocol for employees as per the company policy is as follows:                                 │
│                                                                                                                 │
│  1. **Entitlement**: Sick leave is part of the 21 days of paid leave allotted to all full-time employees on an  │
│  annual basis.                                                                                                  │
│  2. **Eligibility**: New employees must complete their first 30 days of employment to become eligible for sick  │
│  leave.                                                                                                         │
│  3. **Notification Requirement**: Employees are required to inform their manager or designated authority about  │
│  their sick leave on the same day, and this must be done before 10 AM.                                          │
│  4. **Extended Sick Leave**: If an employee needs sick leave exceeding 3 consecutive days, it is mandatory to   │
│  provide supporting documentation, such as a physician’s note or medical certificate, to validate the absence.  │
│  5. **Remote Work Option**: Employees may be allowed to work remotely for up to 3 days a week during minor      │
│  illnesses, but this requires approval from their manager.                                                      │
│                                                                                                                 │
│  This protocol ensures fair use of leave, maintains accountability, and supports employees facing health        │
│  challenges.                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 82f216da-3be5-4c6d-b0fb-30d56d2826a1                                                                     │
│  Agent: HR Policy Assistant                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3e61bef1-aa82-4a2d-91da-99a3f6e0cc16                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The sick leave protocol for employees as per the company policy is as follows:                   │
│                                                                                                                 │
│  1. **Entitlement**: Sick leave is part of the 21 days of paid leave allotted to all full-time employees on an  │
│  annual basis.                                                                                                  │
│  2. **Eligibility**: New employees must complete their first 30 days of employment to become eligible for sick  │
│  leave.                                                                                                         │
│  3. **Notification Requirement**: Employees are required to inform their manager or designated authority about  │
│  their sick leave on the same day, and this must be done before 10 AM.                                          │
│  4. **Extended Sick Leave**: If an employee needs sick leave exceeding 3 consecutive days, it is mandatory to   │
│  provide supporting documentation, such as a physician’s note or medical certificate, to validate the absence.  │
│  5. **Remote Work Option**: Employees may be allowed to work remotely for up to 3 days a week during minor      │
│  illnesses, but this requires approval from their manager.                                                      │
│                                                                                                                 │
│  This protocol ensures fair use of leave, maintains accountability, and supports employees facing health        │
│  challenges.                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The sick leave protocol for employees as per the company policy is as follows:

1. **Entitlement**: Sick leave is part of the 21 days of paid leave allotted to all full-time employees on an annual basis.  
2. **Eligibility**: New employees must complete their first 30 days of employment to become eligible for sick leave.  
3. **Notification Requirement**: Employees are required to inform their manager or designated authority about their sick leave on the same day, and this must be done before 10 AM.  
4. **Extended Sick Leave**: If an employee needs sick leave exceeding 3 consecutive days, it is mandatory to provide supporting documentation, such as a physician’s note or medical certificate, to validate the absence.  
5. **Remote Work Option**: Employees may be allowed to work remotely for up to 3 days a week during minor illnesses, but this requires approval from their manager.  

This protocol ensures fair use of leave, maintains accountability, and supports employees facing health c

In [15]:
pdf_source = PDFKnowledgeSource(file_paths=["meeting_notes.pdf"])

meeting_summarizer = Agent(
    role="Meeting Summary Specialist",
    goal="Extract and summarize action items from past meetings.",
    backstory="You specialize in distilling meetings into concise action points.",
    knowledge_sources=[pdf_source],
    embedder={
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-small",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_key": os.getenv("OPENAI_API_KEY")
        }
    },
    llm=llm,
    verbose=True
)

task = Task(
    description="List the action items from last week's meeting.",
    expected_output="A bullet list of 3–5 action items.",
    agent=meeting_summarizer
)

crew = Crew(
    agents=[meeting_summarizer],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 31beab0f-a82b-42b5-b84a-1545717727eb                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 📚 Retrieved Knowledge ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Additional Information: Weekly Strategy Sync - April 12, 2025                                                  │
│  Participants:                                                                                                  │
│  - CTO, Head of Product, Data Science Lead, Engineering Manager                                                 │
│  Discussion Summary:                                                                                            │
│  - Launch of new AI recommendation engine delayed by 1 week due to testing issues.                              │
│  - Product team to finalize Q2 roadmap by April 20.                                                             │
│  - DS team to evaluate CrewAI as a potential integration partner.                                               │
│  - Engineering to move remaining microservices to Kubernetes by end of May.                                     │
│  Action Items:                                                                                                  │
│  1. Product: Finalize Q2 roadmap (due April 20)                                                                 │
│  2. D...                                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meeting Summary Specialist                                                                              │
│                                                                                                                 │
│  Task: List the action items from last week's meeting.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meeting Summary Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  - Product: Finalize Q2 roadmap (due April 20)                                                                  │
│  - Data Science: Conduct feasibility study on CrewAI (due April 18)                                             │
│  - Engineering: Migrate ML serving infrastructure to Kubernetes (due May 31)                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e4072609-7cfc-4b3d-b4f4-1111dd20c8ba                                                                     │
│  Agent: Meeting Summary Specialist                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 31beab0f-a82b-42b5-b84a-1545717727eb                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: - Product: Finalize Q2 roadmap (due April 20)                                                    │
│  - Data Science: Conduct feasibility study on CrewAI (due April 18)                                             │
│  - Engineering: Migrate ML serving infrastructure to Kubernetes (due May 31)                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Product: Finalize Q2 roadmap (due April 20)  
- Data Science: Conduct feasibility study on CrewAI (due April 18)  
- Engineering: Migrate ML serving infrastructure to Kubernetes (due May 31)


In [17]:
csv_source = CSVKnowledgeSource(file_paths=["feedback.csv"])

feedback_analyst = Agent(
    role="Feedback Analyst",
    goal="Analyze customer sentiment and complaints.",
    backstory="You process raw user feedback and extract recurring themes for the product team.",
    knowledge_sources=[csv_source],
    embedder={
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-small",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_key": os.getenv("OPENAI_API_KEY")
        }
    },
    llm=llm,
    verbose=True
)

task = Task(
    description="What are the top 2 common issues users are facing?",
    expected_output="A summary of top 2 user pain points.",
    agent=feedback_analyst
)

crew = Crew(
    agents=[feedback_analyst],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0d3578bb-d6bb-47b3-8ae8-c10a27563581                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 📚 Retrieved Knowledge ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Additional Information: {PosixPath('knowledge/feedback.csv'): 'id  user_id  comment  rating\n1  101  The UI    │
│  feels slow on mobile  3\n2  102  I love the new dashboard layout!  5\n3  103  Notifications are too frequent   │
│  and not customizable  2\n4  104  Customer support was really helpful  4\n5  105  It’s hard to find the export  │
│  option in reports  3\n'}                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Feedback Analyst                                                                                        │
│                                                                                                                 │
│  Task: What are the top 2 common issues users are facing?                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Feedback Analyst                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The top two recurring issues users are facing, based on the feedback analysis, are:                            │
│                                                                                                                 │
│  1. **Interface and Usability Challenges**: Users find the UI slow on mobile devices (`comment: The UI feels    │
│  slow on mobile`) and also express difficulty in locating essential functions like the export option in         │
│  reports (`comment: It’s hard to find the export option in reports`).                                           │
│                                                                                                                 │
│  2. **Notifications Settings**: Users are frustrated with the frequency of notifications and the lack of        │
│  options to customize them (`comment: Notifications are too frequent and not customizable`).                    │
│                                                                                                                 │
│  These two points highlight performance and usability improvements as critical areas for enhancement.           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8e1b4fce-4df1-4129-ae2d-01354e666058                                                                     │
│  Agent: Feedback Analyst                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0d3578bb-d6bb-47b3-8ae8-c10a27563581                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The top two recurring issues users are facing, based on the feedback analysis, are:              │
│                                                                                                                 │
│  1. **Interface and Usability Challenges**: Users find the UI slow on mobile devices (`comment: The UI feels    │
│  slow on mobile`) and also express difficulty in locating essential functions like the export option in         │
│  reports (`comment: It’s hard to find the export option in reports`).                                           │
│                                                                                                                 │
│  2. **Notifications Settings**: Users are frustrated with the frequency of notifications and the lack of        │
│  options to customize them (`comment: Notifications are too frequent and not customizable`).                    │
│                                                                                                                 │
│  These two points highlight performance and usability improvements as critical areas for enhancement.           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The top two recurring issues users are facing, based on the feedback analysis, are:

1. **Interface and Usability Challenges**: Users find the UI slow on mobile devices (`comment: The UI feels slow on mobile`) and also express difficulty in locating essential functions like the export option in reports (`comment: It’s hard to find the export option in reports`).

2. **Notifications Settings**: Users are frustrated with the frequency of notifications and the lack of options to customize them (`comment: Notifications are too frequent and not customizable`). 

These two points highlight performance and usability improvements as critical areas for enhancement.


In [4]:
json_source = JSONKnowledgeSource(file_paths=["company_info.json"])

llm = LLM(model="gpt-4o")

org_analyst = Agent(
    role="Org Structure Specialist",
    goal="Answer questions about company departments and teams.",
    backstory="You help HR and leadership make decisions based on team sizes and reporting structure.",
    knowledge_sources=[json_source],
    embedder={
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-small",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_key": os.getenv("OPENAI_API_KEY")
        }
    },
    llm=llm
)

task = Task(
    description="List the teams under the Engineering department.",
    expected_output="A list of team names under Engineering.",
    agent=org_analyst
)

crew = Crew(
    agents=[org_analyst],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 42b659f1-8a1f-4b9a-af28-40633a81aed3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 📚 Retrieved Knowledge ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Additional Information: {PosixPath('knowledge/company_info.json'): 'company_name: TechNova Inc.\ndepartments:  │
│  -     name: Product\n    head: Alice\n    teams:       -         name: UX Team\n        members: 5\n\n      -  │
│  name: Analytics Team\n        members: 3\n\n\n\n  -     name: Engineering\n    head: Bob\n    teams:       -   │
│  name: Backend\n        members: 6\n\n      -         name: DevOps\n        members: 4\n\n      -               │
│  name: ML Engineering\n        members: 5\n\n\...                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Org Structure Specialist                                                                                │
│                                                                                                                 │
│  Task: List the teams under the Engineering department.                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Org Structure Specialist                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The teams under the Engineering department at TechNova Inc. are as follows:                                    │
│  - Backend                                                                                                      │
│  - DevOps                                                                                                       │
│  - ML Engineering                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7d795e99-1bde-47f6-aeb9-0f0daf0a515d                                                                     │
│  Agent: Org Structure Specialist                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 42b659f1-8a1f-4b9a-af28-40633a81aed3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The teams under the Engineering department at TechNova Inc. are as follows:                      │
│  - Backend                                                                                                      │
│  - DevOps                                                                                                       │
│  - ML Engineering                                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The teams under the Engineering department at TechNova Inc. are as follows:
- Backend
- DevOps
- ML Engineering


In [11]:
class WeatherKnowledgeSource(BaseKnowledgeSource):
    city: str = Field(description="City for which weather should be fetched")

    def validate_content(self) -> Any:
        pass

    def load_content(self) -> Dict[Any, str]:
        print(f"Fetching weather for {self.city}...")

        # Hardcoded coordinates for demo purposes (San Francisco)
        endpoint = "https://api.open-meteo.com/v1/forecast"
        params = {
                "latitude": 37.77,
                "longitude": -122.42,
                "current_weather": True
            }

        response = requests.get(endpoint, params=params)
        response.raise_for_status()

        weather_data = response.json().get("current_weather", {})
        formatted = self.format_weather(weather_data)
        return {self.city: formatted}

    def format_weather(self, data: dict) -> str:
        if not data:
            return "No weather data available."

        return (
            f"Current weather in {self.city}:\n"
            f"- Temperature: {data.get('temperature')}°C\n"
            f"- Wind Speed: {data.get('windspeed')} km/h\n"
            f"- Weather Code: {data.get('weathercode')}\n"
            f"- Time: {data.get('time')}"
        )

    def add(self) -> None:
        content = self.load_content()
        for _, text in content.items():
            chunks = self._chunk_text(text)
            self.chunks.extend(chunks)

        self._save_documents()


weather_knowledge = WeatherKnowledgeSource(city="San Francisco")

weather_agent = Agent(
    role="Weather Reporter",
    goal="Answer questions about the current weather forecast.",
    backstory="You are a friendly meteorologist who provides real-time weather updates.",
    knowledge_sources=[weather_knowledge],
    embedder={
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-small",
            "api_base": os.getenv("OPENAI_API_BASE"),
            "api_key": os.getenv("OPENAI_API_KEY")
        }
    },
    llm=LLM(model="gpt-4o", temperature=0.0),
    verbose=True
)

task = Task(
    description="What is the current temperature and wind speed in SF?",
    expected_output="A concise weather summary for San Francisco.",
    agent=weather_agent
)

crew = Crew(
    agents=[weather_agent],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 00651807-ef2a-4a1a-99ba-81b148c718d3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Fetching weather for San Francisco...


Output()

╭──────────────────────────────────────────── 📚 Retrieved Knowledge ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Additional Information: Current weather in San Francisco:                                                      │
│  - Temperature: 14.0°C                                                                                          │
│  - Wind Speed: 7.3 km/h                                                                                         │
│  - Weather Code: 1                                                                                              │
│  - Time: 2025-07-18T05:15                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Weather Reporter                                                                                        │
│                                                                                                                 │
│  Task: What is the current temperature and wind speed in SF?                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Weather Reporter                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The current temperature in San Francisco is 14.0°C, with a wind speed of 7.3 km/h. The weather code indicates  │
│  clear conditions as of 5:15 AM on July 18, 2025.                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d355d9be-b69e-4452-b939-05285266e475                                                                     │
│  Agent: Weather Reporter                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 00651807-ef2a-4a1a-99ba-81b148c718d3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The current temperature in San Francisco is 14.0°C, with a wind speed of 7.3 km/h. The weather   │
│  code indicates clear conditions as of 5:15 AM on July 18, 2025.                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The current temperature in San Francisco is 14.0°C, with a wind speed of 7.3 km/h. The weather code indicates clear conditions as of 5:15 AM on July 18, 2025.
